In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from db import ActionDatabase
from runner import process_threads, execute_due_actions

In [40]:
slack_interactor = SlackInteractor()
llm = ClaudeLLM()
action_db = ActionDatabase()

agents = [
    ProjectManagerAgent(llm, action_db, slack_interactor),
    SarcasticAgent(llm, action_db, slack_interactor)
]

In [ ]:
print("\nFetching new messages...")
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(agents, threads)

print("\nChecking for due actions...")
execute_due_actions(agents)


Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 321 old messages
Found 1 new messages
Updated complete_conversations.pkl with new messages
Found 1 threads with new messages.

Processing thread in channel: agentflow
Thread timestamp: 2024-08-18 20:58:43.793899059
Last message content:
Hey everyone, just checking in on this thread :eyes: Wanted to see if there are any open action items that still need to be tackled? Let me know what's left so we can keep things moving forward! :muscle:
XXX ClaudeLLM prompt:

        Analyze the following conversation carefully. Consider the entire thread history when making decisions. Determine if any immediate action is needed or if a delayed task should be scheduled. Consider the following:

        1. Immediate actions: Tasks that need to be done right away based on direct messages to the bot 'agentflow'.
        2. Delayed tasks: Any task that needs to be performed in the future, including check-ins, reminders, or sched